In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%matplotlib inline

import tensorflow as tf
import numpy as np
import pickle as pkl
#from sklearn.manifold import TSNE

from flip_gradient import flip_gradient
from utils import *

In [2]:
source_domain = pkl.load(open('./Datasets/Datael4106/source_domain.pkl', 'rb'))
source_data_separation = 13500
source_train = source_domain['images'][:source_data_separation]
source_train_labels = source_domain['labels'][:source_data_separation]
source_test = source_domain['images'][source_data_separation:]
source_test_labels = source_domain['labels'][source_data_separation:]

target_domain = pkl.load(open('./Datasets/Datael4106/target_domain.pkl', 'rb'))
target_data_separation = int(target_domain['images'].shape[0]*0.9)
target_train = target_domain['images'][:target_data_separation]
target_train_labels = target_domain['labels'][:target_data_separation]
target_test = target_domain['images'][target_data_separation:]
target_test_labels = target_domain['labels'][target_data_separation:]

In [3]:
def doWeightSummary(w):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(w)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(w - mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.histogram('histogram',w)


In [4]:
batch_size = 64

class SuperNovaModel(object):
    def __init__(self):
        self._build_model()
    
    def _build_model(self):
        
        self.X = tf.placeholder(tf.uint8, [None, 21, 21, 3])
        self.y = tf.placeholder(tf.float32, [None, 2])
        self.domain = tf.placeholder(tf.float32, [None, 2])
        self.l = tf.placeholder(tf.float32, [])
        self.train = tf.placeholder(tf.bool, [])
        
        #X_input = (tf.cast(self.X, tf.float32) - pixel_mean) / 255.
        X_input = tf.cast(self.X, tf.float32)
        
        # CNN model for feature extraction
        with tf.variable_scope('feature_extractor'):

            W_conv0 = weight_variable([4, 4, 3, 32])
            b_conv0 = bias_variable([32])
            h_conv0 = tf.nn.relu(conv2d(X_input, W_conv0) + b_conv0)
            
            W_conv1 = weight_variable([3, 3, 32, 32])
            b_conv1 = bias_variable([32])
            h_conv1 = tf.nn.relu(conv2d(h_conv0, W_conv1) + b_conv1)
            
            h_pool0 = max_pool_2x2(h_conv1)
            
            W_conv2 = weight_variable([3, 3, 32, 64])
            b_conv2 = bias_variable([64])
            h_conv2 = tf.nn.relu(conv2d(h_pool0, W_conv2) + b_conv2)
            
            W_conv3 = weight_variable([3, 3, 64, 64])
            b_conv3 = bias_variable([64])
            h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3) + b_conv3)
            
            W_conv4 = weight_variable([3, 3, 64, 64])
            b_conv4 = bias_variable([64])
            h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4) + b_conv4)
            
            h_pool1 = max_pool_2x2(h_conv4)
            
            # The domain-invariant feature
            self.feature = tf.reshape(h_pool1, [-1, 6*6*64])
            
            # Histogram summaries
            doWeightSummary(W_conv0)
            doWeightSummary(W_conv1)
            doWeightSummary(W_conv2)
            doWeightSummary(W_conv3)
            doWeightSummary(W_conv4)
            
        # MLP for class prediction
        with tf.variable_scope('label_predictor',reuse=tf.AUTO_REUSE):
            
            # Switches to route target examples (second half of batch) differently
            # depending on train or test mode.
            all_features = lambda: self.feature
            source_features = lambda: tf.slice(self.feature, [0, 0], [batch_size // 2, -1])
            classify_feats = tf.cond(self.train, source_features, all_features)
            
            all_labels = lambda: self.y
            source_labels = lambda: tf.slice(self.y, [0, 0], [batch_size // 2, -1])
            self.classify_labels = tf.cond(self.train, source_labels, all_labels)
            
            dense0 = tf.layers.dense(inputs=classify_feats,units=6*6*64,activation=tf.nn.relu)
            dropout0 = tf.layers.dropout(inputs=dense0, rate=0.5, training=True)
            d_logits0 = tf.layers.dense(inputs=dropout0, units=64)
            
            dense1 = tf.layers.dense(inputs=d_logits0,units=64,activation=tf.nn.relu)
            dropout1 = tf.layers.dropout(inputs=dense1, rate=0.7, training=True)
            d_logits1 = tf.layers.dense(inputs=dropout1, units=64)
            
            dense2 = tf.layers.dense(inputs=d_logits1,units=64,activation=tf.nn.relu)
            dropout2 = tf.layers.dropout(inputs=dense2, rate=0.7, training=True)
            logits = tf.layers.dense(inputs=dropout2, units=2)
            
            self.pred = tf.nn.softmax(logits)
            self.pred_loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.classify_labels)
            
        # Small MLP for domain prediction with adversarial loss
        with tf.variable_scope('domain_predictor', reuse=tf.AUTO_REUSE):
            
            # Flip the gradient when backpropagating through this operation
            feat = flip_gradient(self.feature, self.l)
            
            dense0 = tf.layers.dense(inputs=feat,units=6*6*64,activation=tf.nn.relu)
            dropout0 = tf.layers.dropout(inputs=dense0, rate=0.7, training=True)
            d_logits0 = tf.layers.dense(inputs=dropout0, units=64)
            
            dense1 = tf.layers.dense(inputs=d_logits0,units=64,activation=tf.nn.relu)
            dropout1 = tf.layers.dropout(inputs=dense1, rate=0.7, training=True)
            d_logits = tf.layers.dense(inputs=dropout1, units=2)
            
            self.domain_pred = tf.nn.softmax(d_logits)
            self.domain_loss = tf.nn.softmax_cross_entropy_with_logits(logits=d_logits, labels=self.domain)

In [5]:
# Build the model graph
graph = tf.get_default_graph()
with graph.as_default():
    model = SuperNovaModel()
    
    learning_rate = tf.placeholder(tf.float32, [])
    
    pred_loss = tf.reduce_mean(model.pred_loss)
    domain_loss = tf.reduce_mean(model.domain_loss)
    total_loss = pred_loss + domain_loss
    
    tf.summary.scalar("source_accuracy", pred_loss)
    tf.summary.scalar("domain_accuracy", domain_loss)

    regular_train_op = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(pred_loss)
    dann_train_op = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(total_loss)
    
    # Evaluation
    correct_label_pred = tf.equal(tf.argmax(model.classify_labels, 1), tf.argmax(model.pred, 1))
    label_acc = tf.reduce_mean(tf.cast(correct_label_pred, tf.float32))
    correct_domain_pred = tf.equal(tf.argmax(model.domain, 1), tf.argmax(model.domain_pred, 1))
    domain_acc = tf.reduce_mean(tf.cast(correct_domain_pred, tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
classes = 2
def fix_label_dimension(labels):
    the_batch = labels.shape[0]
    reshaped = np.zeros(the_batch*classes).reshape(the_batch,classes)
    for i in range(the_batch):
        val = 0
        if labels[i] == 1:
            val = 1
        reshaped[i][val] = 1
    return reshaped


In [10]:
BASE_LOG_DIR = "./logs/"
TARGET_DIR = "target/"
SOURCE_DIR = "source/"

def train_and_evaluate(training_mode, graph, model, num_steps=8600, verbose=False):
    """Helper to run the model with different training modes."""

    with tf.Session(graph=graph) as sess:
        
        # TensorBoard
        directory = BASE_LOG_DIR
        if training_mode == 'dann':
            directory = BASE_LOG_DIR + TARGET_DIR
        elif training_mode == 'source':
            directory = BASE_LOG_DIR + SOURCE_DIR
        writer = tf.summary.FileWriter(directory, sess.graph)
        merged = tf.summary.merge_all()
        
        tf.global_variables_initializer().run()

        # Batch generators
        gen_source_batch = batch_generator(
            [source_train, source_train_labels], batch_size // 2)
        gen_target_batch = batch_generator(
            [target_train, target_train_labels], batch_size // 2)
        gen_source_only_batch = batch_generator(
            [source_train, source_train_labels], batch_size)
        gen_target_only_batch = batch_generator(
            [target_train, target_train_labels], batch_size)

        domain_labels = np.vstack([np.tile([1., 0.], [batch_size // 2, 1]),
                                   np.tile([0., 1.], [batch_size // 2, 1])])

        # Training loop
        for i in range(num_steps):
            
            # Adaptation param and learning rate schedule as described in the paper
            p = float(i) / num_steps
            gamma = 25.
            l = 2. / (1. + np.exp(-gamma * p)) - 1
            lr = 0.01 / (1. + 10 * p)**0.75

            # Training step
            if training_mode == 'dann':

                X0, y0 = next(gen_source_batch)
                X1, y1 = next(gen_target_batch)
                y0 = fix_label_dimension(y0)
                y1 = fix_label_dimension(y1)
                X = np.vstack([X0, X1])
                y = np.vstack([y0, y1])

                _, batch_loss, dloss, ploss, d_acc, p_acc = sess.run(
                    [dann_train_op, total_loss, domain_loss, pred_loss, domain_acc, label_acc],
                    feed_dict={model.X: X, model.y: y, model.domain: domain_labels,
                               model.train: True, model.l: l, learning_rate: lr})

                if verbose and i % 100 == 0:
                    print('loss: {}  d_acc: {}  p_acc: {}  p: {}  l: {}  lr: {}'.format(
                            batch_loss, d_acc, p_acc, p, l, lr))

            elif training_mode == 'source':
                X, y = next(gen_source_only_batch)
                y = fix_label_dimension(y)
                _, batch_loss = sess.run([regular_train_op, pred_loss],
                                     feed_dict={model.X: X, model.y: y, model.train: False,
                                                model.l: l, learning_rate: lr})

            elif training_mode == 'target':
                X, y = next(gen_target_only_batch)
                y = fix_label_dimension(y)
                _, batch_loss = sess.run([regular_train_op, pred_loss],
                                     feed_dict={model.X: X, model.y: y, model.train: False,
                                                model.l: l, learning_rate: lr})

            if i % 50 == 0:
                
                summary = sess.run(merged,feed_dict={model.X: target_test, model.y: fix_label_dimension(target_test_labels),
                                       model.train: False})
        
                writer.add_summary(summary,i)
        # Compute final evaluation on test data
        source_acc = sess.run(label_acc,
                            feed_dict={model.X: source_test, model.y: fix_label_dimension(source_test_labels),
                                       model.train: False})

        target_acc = sess.run(label_acc,
                            feed_dict={model.X: target_test, model.y: fix_label_dimension(target_test_labels),
                                       model.train: False})
        """
        test_domain_acc = sess.run(domain_acc,
                            feed_dict={model.X: combined_test_imgs,
                                       model.domain: combined_test_domain, model.l: 1.0})
        
        test_emb = sess.run(model.feature, feed_dict={model.X: combined_test_imgs})
        """
        
    return source_acc, target_acc
    #return source_acc, target_acc, test_domain_acc, test_emb

"""

print('\nSource only training')
source_acc, target_acc, _, source_only_emb = train_and_evaluate('source', graph, model)
print('Source (Resnet Super Nova) accuracy:', source_acc)
print('Target (Real Super Nova) accuracy:', target_acc)
"""

"""
print('\nSource only training')
source_acc, target_acc = train_and_evaluate('source', graph, model)
print('Source (Resnet Super Nova) accuracy:', source_acc)
print('Target (Real Super Nova) accuracy:', target_acc)
"""
print('\nDomain adaptation training')
source_acc, target_acc = train_and_evaluate('dann', graph, model,100,True)
print('Source (Resnet Super Nova) accuracy:', source_acc)
print('Target (Real Super Nova) accuracy:', target_acc)
#print('Domain accuracy:', d_acc)


Domain adaptation training


InvalidArgumentError: tags and values not the same shape: [] != [64] (tag 'domain_accuracy')
	 [[{{node domain_accuracy}} = ScalarSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](domain_accuracy/tags, domain_predictor/softmax_cross_entropy_with_logits_sg/_81)]]

Caused by op 'domain_accuracy', defined at:
  File "C:\Program Files\Python3.6\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files\Python3.6\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\dario\downloads\gitprojects\tf-dann\venv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\dario\downloads\gitprojects\tf-dann\venv\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\dario\downloads\gitprojects\tf-dann\venv\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File